In [2]:
from graphviz import Source

data = """
digraph G {
    graph [rankdir=LR labelfontcolor=red fontname="monospace" nodesep=0.6 size="10,10"]
    node [fontname="monospace" fontsize=11]
    edge [fontname="monospace" color="grey" fontsize=11]
    
    S0 [label="start" shape="circle", pos="1,1"]
    S1 [label="S1 (generate)" shape="circle", pos="2,1"]
    S2 [label="S2 (hold)" shape="circle", pos="3,1"]
    S3 [label="S3 (fill)" shape="circle", pos="4,1"]
    S4 [label="S4 (hold)" shape="doublecircle", pos="5,1"]

    S0 -> S1 [label="h > h_min", constraint=false]
    S1 -> S2 [label="n11 >= n11_max", constraint=true]
    S2 -> S3 [label="h <= 0", constraint=true]
    S3 -> S4 [label="h >= 0", constraint=true]
    S4 -> S1 [label="h > h_min", constraint=false]
}
"""
gv = Source(data)
gv.format='svg'
gv.render( 'EbbOperation' )


ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tsvg', '-O', 'EbbOperation'], make sure the Graphviz executables are on your systems' PATH

In [2]:
def prime(fn):
    def wrapper(*args, **kwargs):
        v = fn(*args, **kwargs)
        v.send(None)
        return v
    return wrapper

In [13]:
class RegexFSM:
    def __init__(self):
        self.start = self._create_start()
        self.q1 = self._create_q1()
        self.q2 = self._create_q2()
        self.q3 = self._create_q3()
        
        self.current_state = self.start
        self.stopped = False
        
    def send(self, char):
        try:
            self.current_state.send(char)
        except StopIteration:
            self.stopped = True
        
    def does_match(self):
        if self.stopped:
            return False
        return self.current_state == self.q3

    @prime
    def _create_start(self):
        while True:
            char = yield
            if char == 'a':
                self.current_state = self.q1
            else:
                break
    
    @prime
    def _create_q1(self):
        while True:
            char = yield
            if char == 'b':
                self.current_state = self.q2
            elif char == 'c':
                self.current_state = self.q3
            else:
                break

    @prime
    def _create_q2(self):
        while True:
            char = yield
            if char == 'b':
                self.current_state = self.q2
            elif char == 'c':
                self.current_state = self.q3
            else:
                break

    @prime
    def _create_q3(self):
        while True:
            char = yield
            break

In [14]:
def grep_regex(text):
    evaluator = RegexFSM()
    for ch in text:
        evaluator.send(ch)
    return evaluator.does_match()

In [15]:
grep_regex("a")

False

In [16]:
grep_regex("ab")

False

In [17]:
grep_regex("ac")

True

In [18]:
grep_regex("abc")

True

In [19]:
grep_regex("aba")

False

In [20]:
grep_regex("abbbbbbbc")

True

In [21]:
grep_regex("abcc")

False

In [22]:
grep_regex("abcd")

False

In [23]:
grep_regex("bcbc")

False